# **Stroke prediction**
### Fully refered by Ahmed Ashour and some more new things add in the code and executions,Its absolutely usefull for me 
 
### 0. Import necessary library
### 1. cleaning 
### 2. preprocessing
### 3. feature engineering and selection 
### 4. classifiers

# Library

In [ ]:
                                                                                                                                            import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import plotly
import plotly.express as ex
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
pyo.init_notebook_mode()
from imblearn.over_sampling import SMOTE


In [ ]:
import scikitplot as skplt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.svm import SVC

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.rc('figure',figsize= (17,13))
sns.set_context('paper',font_scale=2)

In [ ]:
#data preprocessing
data = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
data.head()

In [ ]:
plt.title('Missing value status',fontweight = 'bold')
ax = sns.heatmap(data.isnull().sum().to_frame(),annot=True,fmt = 'd',cmap = 'vlag')
ax.set_xlabel('Amount Missing')
plt.show()

In [ ]:
data.shape

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

# Variance features Distribution

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
data.plot(kind="hist", y="age", bins=70, color="k", ax=axes[0][0])
data.plot(kind="hist", y="bmi", bins=100, color="b", ax=axes[0][1])
data.plot(kind="hist", y="heart_disease", bins=6, color="k", ax=axes[1][0])
data.plot(kind="hist", y="avg_glucose_level", bins=100, color="orange", ax=axes[1][1])
plt.show()

In [ ]:
labels =data['stroke'].value_counts(sort = True).index
sizes = data['stroke'].value_counts(sort = True)

colors = ["b","red"]
explode = (0.05,0) 
 
plt.figure(figsize=(7,7))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=90,)

plt.title('Customer Churn Breakdown')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.countplot('stroke', hue='gender', data=data, palette='Reds')

In [ ]:
#Correlation process
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
data.plot(kind='scatter', x='age', y='avg_glucose_level', alpha=0.5, color='k', ax=axes[0], title="Age vs. avg_glucose_level")
data.plot(kind='scatter', x='bmi', y='avg_glucose_level', alpha=0.5, color='orange', ax=axes[1], title="bmi vs. avg_glucose_level")

plt.show()

In [ ]:
sns.set(style="ticks");
pal = ["#FA5858", "#58D3F7"]

sns.pairplot(data, hue="stroke", palette=pal);
plt.title("stroke");

In [ ]:
sns.heatmap(data.corr(),annot=True,cmap="RdYlGn")

In [ ]:
#Boxplot
sns.boxplot(data=data,x=data["bmi"],color='r');

In [ ]:
bmi_outliers=data.loc[data['bmi']>50]
bmi_outliers['bmi'].shape

In [ ]:
# mean with outliers 
print(bmi_outliers['stroke'].value_counts())

In [ ]:
data["bmi"] = data["bmi"].apply(lambda x: 50 if x>50 else x)
data["bmi"] = data["bmi"].fillna(28.4)

In [ ]:
print ("\nMissing values :  ", data.isnull().sum().values.sum())

In [ ]:
cat_df = data[['gender','Residence_type','smoking_status','stroke']]
summary = pd.concat([pd.crosstab(cat_df[x], cat_df.stroke) for x in cat_df.columns[:-1]], keys=cat_df.columns[:-1])
summary

In [ ]:
#marriage people have a stroke is possible comparison
plt.figure(figsize= (13,6))
stroke = data.loc[data['stroke']==1]
sns.countplot(data= stroke,x = 'ever_married',palette = 'inferno')

In [ ]:
#work types people
sns.countplot(data=stroke,x= 'work_type',palette = 'inferno');

In [ ]:
#smoker comparison
sns.countplot(data=stroke,x='smoking_status',palette='autumn');

In [ ]:
#urban vs rural
sns.countplot(data=stroke,x='Residence_type',palette='Greens');

In [ ]:
#strokes vs heartdisease
sns.countplot(data=stroke,x='heart_disease',palette='cool');

In [ ]:
sns.countplot(data=stroke,x='hypertension',palette='Pastel2');

### ITs above comparison show the what is impact of above ?
    Avg glucose level is high with old people
    BMI >40 have low avg glucose
    Being unmarried reduces your risk of a stroke
    Being a smoker or a formerly smoker increases your risk of having a stroke
    more than 25% of strok cases They had hypertension


### Data preprocessing

#### Encoding Categorical Features

In [ ]:
data["Residence_type"] = data["Residence_type"].apply(lambda x: 1 if x=="Urban" else 0)
data["ever_married"] = data["ever_married"].apply(lambda x: 1 if x=="Yes" else 0)
data["gender"] = data["gender"].apply(lambda x: 1 if x=="Male" else 0)

 
data = pd.get_dummies(data=data, columns=['smoking_status'])
data = pd.get_dummies(data=data, columns=['work_type'])

In [ ]:
data

In [ ]:
data.shape

### Scaling The variance in Features

In [ ]:
std=StandardScaler()
columns = ['avg_glucose_level','bmi','age']
scaled = std.fit_transform(data[['avg_glucose_level','bmi','age']])
scaled = pd.DataFrame(scaled,columns=columns)
data=data.drop(columns=columns,axis=1)

In [ ]:
data=data.merge(scaled, left_index=True, right_index=True, how = "left")
data

In [ ]:
data = data.drop(columns = 'id',axis=1)
data[data.isnull().any(axis=1)]

### Classifier

In [ ]:
X = data.drop(['stroke'], axis=1).values 
y = data['stroke'].values

In [ ]:
#spliting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
#adaboost classification
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(),n_estimators =100, learning_rate = 0.5, random_state = 100)

ada_clf.fit(X_train,y_train)
ada_pred = ada_clf.predict(X_train)
print('predict train',ada_pred)
print('parms',ada_clf.get_params)

ada_clf_score = ada_clf.score(X_test,y_test)
print('mean accuracy:%0.2f' %ada_clf.score(X_test,y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
#xgboost
xgboost = GradientBoostingClassifier(random_state=0)
xgboost.fit(X_train, y_train)
#== 
#Score 
#== 
xgboost_score = xgboost.score(X_train, y_train)
xgboost_test = xgboost.score(X_test, y_test)

In [ ]:
#testing model 
#== 
y_pred = xgboost.predict(X_test)
#== 
#evaluation
#== 
cm = confusion_matrix(y_test,y_pred)
print('Training Score',xgboost_score)
print('Testing Score \n',xgboost_test)

#=== 
#Confusion Matrix 
plt.figure(figsize=(14,5))

conf_matrix = pd.DataFrame(data=cm,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="Reds");
print(accuracy_score(y_test,y_pred))

In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print('Testing Score \n',score)
logistic_score = model.score(X_train, y_train)
logistic_test = model.score(X_test, y_test)
y_pred= model.predict(X_test)
print(classification_report(y_test, y_pred))
#== 
cm = confusion_matrix(y_test,y_pred)
print(cm)

# Thanks you